In [1]:
%pip install gradio==3.50.2 opencv-python torch torchvision Pillow ultralytics git+https://github.com/openai/CLIP.git


Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/openai/CLIP.git to /private/var/folders/14/7jpfkb0s2tj8424k7bzwb2qw0000gn/T/pip-req-build-86p6h1hw
  Running command git clone -q https://github.com/openai/CLIP.git /private/var/folders/14/7jpfkb0s2tj8424k7bzwb2qw0000gn/T/pip-req-build-86p6h1hw
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [5]:
from ultralytics import YOLOWorld
import numpy as np
from PIL import Image
import gradio as gr

model = YOLOWorld('yolov8l-world.pt')

def detect_objects(image, prompt_text, conf_threshold):
    if image is None:
        return None, "이미지를 입력해주세요."
    if not prompt_text.strip():
        return None, "탐지할 대상을 입력해주세요."

    img = np.array(image)
    model.set_classes(prompt_text.split(','))
    results = model.predict(img, conf=conf_threshold, verbose=False)

    annotated = results[0].plot()
    annotated_img = Image.fromarray(annotated)

    summary = []
    for box in results[0].boxes:
        cls_id = int(box.cls)
        conf = float(box.conf)
        label = model.names.get(cls_id, "unknown")
        summary.append(f"{label} ({conf:.2f})")
    summary_text = "\n".join(summary) if summary else "탐지된 객체 없음"

    return annotated_img, summary_text


In [6]:
# YOLO-World Gradio (for Gradio v3.x - webcam 지원)

def yolo_world_interface(image, prompt_text, conf_threshold):
    # 이미지가 None이거나 비어있는 경우 처리
    if image is None:
        return None, "이미지를 입력해주세요. 웹캠 버튼을 눌러 사진을 찍어주세요."
    return detect_objects(image, prompt_text, conf_threshold)

with gr.Blocks(title="YOLO-World Webcam Detection") as demo:
    gr.Markdown("## YOLO-World 실시간 객체 탐지기 \n웹캠 버튼을 눌러 사진을 찍고, 텍스트로 원하는 객체를 입력한 후 탐지 버튼을 누르세요.")

    with gr.Row():
        with gr.Column():
            image_in = gr.Image(label="📷 Webcam Input (사진 찍기 버튼 클릭)", source="webcam", streaming=False, type="pil")
            prompt_in = gr.Textbox(label="🔍 탐지할 객체 (예: person, eyeglass)", value="person, eyeglass, nose, mouth")
            conf_in = gr.Slider(0.1, 1.0, value=0.5, step=0.05, label="Confidence")
            detect_btn = gr.Button("🔍 탐지 시작", variant="primary")

        with gr.Column():
            image_out = gr.Image(label="탐지 결과")
            summary_out = gr.Textbox(label="탐지 요약")

    # ✅ 버튼 클릭 시 현재 웹캠 이미지로 탐지 실행
    detect_btn.click(
        yolo_world_interface,
        inputs=[image_in, prompt_in, conf_in],
        outputs=[image_out, summary_out]
    )

# ✅ Gradio 3.x에서는 source="webcam" (문자열), share=True로 외부 브라우저 접근, queue 활성화
demo.queue()
demo.launch(share=True, debug=True)


Running on local URL:  http://127.0.0.1:7860
IMPORTANT: You are using gradio version 3.50.2, however version 4.44.1 is available, please upgrade.
--------
Running on public URL: https://417e6086bf9ed6a650.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://417e6086bf9ed6a650.gradio.live
